# Mask RCNN - train lunar crater dataset
Zhiang Chen, Sep 16, 2018
#### 1. import libraries

In [1]:
import os
import sys
import random
import math
import time
import numpy as np
import cv2
import matplotlib.pyplot as plt

from mrcnn import utils
import mrcnn.model as modellib
from mrcnn import visualize
from mrcnn.model import log

import crater

Using TensorFlow backend.


#### 2. import dataset

In [2]:
config = crater.CraterConfig()
config.display()

train_dataset = crater.CraterDataset()
valid_dataset = crater.CraterDataset()

def load_dataset(dataset, subset):
    assert subset in ['train', 'val']
    
    if subset is 'train':
        for i in range(10):
            dataset.load_crater('../../dataset/lunar_craters', 'train', str(i))
    else:
        for i in range(42, 46):
            dataset.load_crater('../../dataset/lunar_craters', 'val', str(i))
    

load_dataset(train_dataset, 'train')
load_dataset(valid_dataset, 'val')

train_dataset.prepare()
valid_dataset.prepare()


Configurations:
BACKBONE                       resnet101
BACKBONE_STRIDES               [4, 8, 16, 32, 64]
BATCH_SIZE                     1
BBOX_STD_DEV                   [0.1 0.1 0.2 0.2]
COMPUTE_BACKBONE_SHAPE         None
DETECTION_MAX_INSTANCES        600
DETECTION_MIN_CONFIDENCE       0.9
DETECTION_NMS_THRESHOLD        0.3
FPN_CLASSIF_FC_LAYERS_SIZE     1024
GPU_COUNT                      1
GRADIENT_CLIP_NORM             5.0
IMAGES_PER_GPU                 1
IMAGE_CHANNEL                  3
IMAGE_MAX_DIM                  256
IMAGE_META_SIZE                14
IMAGE_MIN_DIM                  256
IMAGE_MIN_SCALE                0
IMAGE_RESIZE_MODE              square
IMAGE_SHAPE                    [256 256   3]
LEARNING_MOMENTUM              0.9
LEARNING_RATE                  0.001
LOSS_WEIGHTS                   {'rpn_bbox_loss': 1.0, 'mrcnn_class_loss': 1.0, 'mrcnn_mask_loss': 1.0, 'mrcnn_bbox_loss': 1.0, 'rpn_class_loss': 1.0}
MASK_POOL_SIZE                 14
MASK_SHAPE             

#### 3. create network model

In [3]:
ROOT_DIR = os.path.abspath("../../")
MODEL_DIR = os.path.join(ROOT_DIR, "logs")
COCO_MODEL_PATH = os.path.join(ROOT_DIR, "mask_rcnn_coco.h5")

In [4]:
# the model will be saved under ../../logs
model = modellib.MaskRCNN(mode="training", config=config,
                          model_dir=MODEL_DIR)

In [5]:
# Which weights to start with?
init_with = "coco"  # imagenet, coco, or last

if init_with == "imagenet":
    model.load_weights(model.get_imagenet_weights(), by_name=True)
elif init_with == "coco":
    # Load weights trained on MS COCO, but skip layers that
    # are different due to the different number of classes
    # See README for instructions to download the COCO weights
    model.load_weights(COCO_MODEL_PATH, by_name=True,
                       exclude=["mrcnn_class_logits", "mrcnn_bbox_fc", 
                                "mrcnn_bbox", "mrcnn_mask"])
elif init_with == "last":
    # Load the last model you trained and continue training
    model.load_weights(model.find_last(), by_name=True)

#### 4. training model

In [6]:
model.train(train_dataset, valid_dataset, 
            learning_rate=config.LEARNING_RATE, 
            epochs=1, 
            layers='heads')


Starting at epoch 0. LR=0.001

Checkpoint Path: /models/Mask_RCNN/logs/crater20180917T0326/mask_rcnn_crater_{epoch:04d}.h5
Selecting layers to train
fpn_c5p5               (Conv2D)
fpn_c4p4               (Conv2D)
fpn_c3p3               (Conv2D)
fpn_c2p2               (Conv2D)
fpn_p5                 (Conv2D)
fpn_p2                 (Conv2D)
fpn_p3                 (Conv2D)
fpn_p4                 (Conv2D)
In model:  rpn_model
    rpn_conv_shared        (Conv2D)
    rpn_class_raw          (Conv2D)
    rpn_bbox_pred          (Conv2D)
mrcnn_mask_conv1       (TimeDistributed)
mrcnn_mask_bn1         (TimeDistributed)
mrcnn_mask_conv2       (TimeDistributed)
mrcnn_mask_bn2         (TimeDistributed)
mrcnn_class_conv1      (TimeDistributed)
mrcnn_class_bn1        (TimeDistributed)
mrcnn_mask_conv3       (TimeDistributed)
mrcnn_mask_bn3         (TimeDistributed)
mrcnn_class_conv2      (TimeDistributed)
mrcnn_class_bn2        (TimeDistributed)
mrcnn_mask_conv4       (TimeDistributed)
mrcnn_mask_bn4

/usr/local/lib/python3.5/dist-packages/tensorflow/python/ops/gradients_impl.py:108: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "
/usr/local/lib/python3.5/dist-packages/keras/engine/training_generator.py:46: UserWarning: Using a generator with `use_multiprocessing=True` and multiple workers may duplicate your data. Please consider using the`keras.utils.Sequence class.
  UserWarning('Using a generator with `use_multiprocessing=True`'


Epoch 1/1
100/100 [==============================] - 98s 980ms/step - loss: 2.6232 - rpn_class_loss: 0.5003 - rpn_bbox_loss: 1.0699 - mrcnn_class_loss: 0.0357 - mrcnn_bbox_loss: 0.6655 - mrcnn_mask_loss: 0.3518 - val_loss: 2.0012 - val_rpn_class_loss: 0.3168 - val_rpn_bbox_loss: 0.7737 - val_mrcnn_class_loss: 0.0243 - val_mrcnn_bbox_loss: 0.5821 - val_mrcnn_mask_loss: 0.3043


In [ ]:
model.train(train_dataset, valid_dataset, 
            learning_rate=config.LEARNING_RATE / 10,
            epochs=200, 
            layers="all")


Starting at epoch 1. LR=0.0001

Checkpoint Path: /models/Mask_RCNN/logs/crater20180917T0326/mask_rcnn_crater_{epoch:04d}.h5
Selecting layers to train
conv1                  (Conv2D)
bn_conv1               (BatchNorm)
res2a_branch2a         (Conv2D)
bn2a_branch2a          (BatchNorm)
res2a_branch2b         (Conv2D)
bn2a_branch2b          (BatchNorm)
res2a_branch2c         (Conv2D)
res2a_branch1          (Conv2D)
bn2a_branch2c          (BatchNorm)
bn2a_branch1           (BatchNorm)
res2b_branch2a         (Conv2D)
bn2b_branch2a          (BatchNorm)
res2b_branch2b         (Conv2D)
bn2b_branch2b          (BatchNorm)
res2b_branch2c         (Conv2D)
bn2b_branch2c          (BatchNorm)
res2c_branch2a         (Conv2D)
bn2c_branch2a          (BatchNorm)
res2c_branch2b         (Conv2D)
bn2c_branch2b          (BatchNorm)
res2c_branch2c         (Conv2D)
bn2c_branch2c          (BatchNorm)
res3a_branch2a         (Conv2D)
bn3a_branch2a          (BatchNorm)
res3a_branch2b         (Conv2D)
bn3a_branch2b

/usr/local/lib/python3.5/dist-packages/tensorflow/python/ops/gradients_impl.py:108: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "
/usr/local/lib/python3.5/dist-packages/keras/engine/training_generator.py:46: UserWarning: Using a generator with `use_multiprocessing=True` and multiple workers may duplicate your data. Please consider using the`keras.utils.Sequence class.
  UserWarning('Using a generator with `use_multiprocessing=True`'


Epoch 2/200
100/100 [==============================] - 120s 1s/step - loss: 1.9769 - rpn_class_loss: 0.3670 - rpn_bbox_loss: 0.8681 - mrcnn_class_loss: 0.0291 - mrcnn_bbox_loss: 0.4048 - mrcnn_mask_loss: 0.3079 - val_loss: 1.6268 - val_rpn_class_loss: 0.2152 - val_rpn_bbox_loss: 0.7231 - val_mrcnn_class_loss: 0.0169 - val_mrcnn_bbox_loss: 0.3902 - val_mrcnn_mask_loss: 0.2813
Epoch 3/200
100/100 [==============================] - 96s 963ms/step - loss: 2.0244 - rpn_class_loss: 0.3869 - rpn_bbox_loss: 0.8758 - mrcnn_class_loss: 0.0346 - mrcnn_bbox_loss: 0.4108 - mrcnn_mask_loss: 0.3163 - val_loss: 1.8766 - val_rpn_class_loss: 0.3254 - val_rpn_bbox_loss: 0.8067 - val_mrcnn_class_loss: 0.0302 - val_mrcnn_bbox_loss: 0.4159 - val_mrcnn_mask_loss: 0.2984
Epoch 4/200
100/100 [==============================] - 96s 961ms/step - loss: 1.8656 - rpn_class_loss: 0.3217 - rpn_bbox_loss: 0.8121 - mrcnn_class_loss: 0.0318 - mrcnn_bbox_loss: 0.3942 - mrcnn_mask_loss: 0.3058 - val_loss: 1.7835 - val_rpn_